# **Elliptic curves cryptography**

The package provides:
* Elliptic curve cryptography algorithms
* Elliptic curve algebra with different forms of curves
* Some cryptanalysis algorithms
This notebook illustrates the above with simple examples. The code is compete enought to cover the functionality extensively.

Contents
--------
1. [Basic usage](#basic_usage)
    1. [Fetching standard curves](#get_curve)
    2. [Elliptic Curve Digital Signature Algorithm (ECDSA)](#ecdsa)
    3. [Edwards Curve Digital Signature Algorithm (EdDSA)](#eddsa)
    4. [Elliptic Curve Diffie-Hellman (ECDH)](#ecdh)
2. [Custom elliptic curves](#custom_curves)
3. [Cryptographic algorithms with custom curves](#custom_algos)
4. [Cryptanalysis](#cryptanalysis)
    1. [Pohlig-Hellman algorithm](#pohlig_hellman)
    2. [Partially known nonces attack](#hnp)
5. [Tests](#tests)


## <a name="basic_usage"></a>Basic usage

We start with basic usage of the package. Importing the package exposes the following functionality:
* Standard elliptic curves fetching and manipulation
* Signatures schemes (ECDSA, EdDSA)
* Secret sharing (ECDH)

 

In [1]:
import random
import ecc

### <a name="get_curve"></a>Fetching standard curves

Standard elliptic curves can be fetched with the function `get_curve` which returns an `Elliptic_Curve` instance, along with other characteristics relevant to cryptographic applications: the subgroup generator, the subgroup order and the cofactor. The description of curve operations and other functionality of `Elliptic_Curve` objects is given in section [Custom elliptic curves](#custom_curves).

In [2]:
elliptic_curve, generator, subgroup_order, cofactor = ecc.get_curve("Secp256k1")
print(f"elliptic curve: {elliptic_curve}\nGenerator: {generator}\n"
      f"Subgroup order: {subgroup_order}\nCofactor: {cofactor}")

elliptic curve: Weierstrass curve y^2 = x^3 + ax + b (mod p)
p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
a = 0x0
b = 0x7
Generator: (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)
Subgroup order: 115792089237316195423570985008687907852837564279074904382605163141518161494337
Cofactor: 1


### <a name="ecdsa"></a>Elliptic Curve Digital Signature Algorithm (ECDSA)

The ECDSA signature scheme is described in [RFC 6090](https://www.rfc-editor.org/rfc/rfc6090) and [RFC 6979](https://www.rfc-editor.org/rfc/rfc6979). ECDSA is built from the combination of an elliptic curve and a hash function.

Standard ECDSA protocols are instantiated through the `ECDSA` class, with a standard curve name (one available with `get_curve`) and a standard hash function. In contrast with, for instance, EdDSA, the RFCs are not strict about valid hash/curves combinations nor data encoding. This reflects in the data types returned by various methods: a pair of `int` for signatures, a curve point for public keys, etc. These would have to be appropriately encoded before transmission in practical applications.

In [3]:
# Instantiate bitcoin ECDSA signature scheme
ecdsa = ecc.ECDSA(curve_name="Secp256k1", hash_name="sha256")
# Generate secret integer between 2 and the group order
secret = random.randint(2, ecdsa.n-1) # not secure for cryptographic applications
# and a public key
public_key = ecdsa.public_key(secret) # pair of int

public_key

(58128564310083070330275462878938174259287534296167972427341594703271318455013,
 73737404552307517624652646873051671683881260560077967258375024182685707525051)

In [4]:
# Sign a message
message = b"A first message"
signature = ecdsa.sign(message, secret) # returned as a tuple of int

signature

(29785879176329441955487661916420389239884272275590184132838508833139586876041,
 99663528411773968178105414283327223456613060648822441000722394008568783479630)

In [5]:
# Verify the signature
ecdsa.verify(message, signature, public_key)

True

The deterministic nonce generation algorithm of RFC 6979 is implemented and is the default, but can be overridden with the `nonce` parameter.

In [6]:
# RFC 6979 specifies deterministic nonce generation to avoid reuse...
nonce = next(ecdsa.nonce_gen(message, secret))
# ...otherwise secret recovery is possible
message2 = b"A second message"
signature2 = ecdsa.sign(message2, secret, nonce=nonce)
sk, k = ecdsa.secret_recovery(message, signature, message2, signature2)

k == nonce, sk == secret

(True, True)

### <a name="eddsa"></a>Edwards Curve Digital Signature Algorithm (EdDSA)

This signature scheme is given in [RFC 8032](https://www.rfc-editor.org/rfc/rfc8032), with two instances described: Ed25519 and ED448, using definite pairs of curves/hash functions. The scheme itself differs slightly from ECDSA (and is secure against nonce reuse).

EdDSA instances are created by calling `eddsa_obj` with the name of the instance as the only parameter (`'Ed25519'` or `'Ed448'`). Contrary to ECDSA, data encoding is specified, which reflects in the returned values.

In [7]:
eddsa = ecc.eddsa_obj("Ed25519")
secret = random.randbytes(32) # RFC 8032 requires a 32-bytes string for edwards25519
pubkey = eddsa.public_key(secret) # encoded as a byte string

In [8]:
# sign a message
message = "Important message".encode("utf-8")
signature = eddsa.sign(message, secret) # returned as a byte string

In [9]:
signature.hex()

'092437c6c92f16a74dcff8744ce8cc3711a44ef434a1f1fa067fa2bf21706b4d963c6f93a31feb02611113f702729287ef33f35c36466c0d5e198b6704cce309'

In [10]:
# Verify the signature
eddsa.verify(message, signature, pubkey)

True

### <a name="ecdh"></a>Elliptic Curve Diffie-Hellman (ECDH)

The ECDH key exchange protocol is described in [RFC 7748](https://www.rfc-editor.org/rfc/rfc7748). The RFC specifies two instances: X25519 and X448.

EdDSA instances are created with by calling `ecdh_obj` with the name of the instance as the only parameter (`'X25519'` or `'X448'`). For these instances, data encoding is specified, and the (signature, public key) values are returned as byte strings. Moreover, the standard instances use secret keys encoded as byte strings, with internal conversion to integer. However, such conversion would not be compatible with custom instances (using different elliptic curves). Thus integer secret keys are also accepted for the sake of generality.

In [11]:
ecdh = ecc.ecdh_obj("X25519")
secretA = random.randbytes(32) # standard instances accept secret keys as byte strings, here 32 bytes for X25519
pubkeyA = ecdh.public_key(secretA) # encoded as a byte string
secretB = random.randint(2, ecdh.n-1) # secret keys as integers are also accepted for broader compatibility
pubkeyB = ecdh.public_key(secretB)
shared_secret = ecdh.ecdh(secretA, pubkeyB, use_cofactor=False) # returned as a byte string

In [12]:
shared_secret.hex()

'cb976a3dd7353f57af6adfcd92d5c6637fa71fdb8cefbcb2c10f5b3fa0adfb47'

In [13]:
shared_secret == ecdh.ecdh(secretB, pubkeyA, use_cofactor=False)

True

The [Menezes-Qu-Vanstone](https://en.wikipedia.org/wiki/MQV) (MQV) signature scheme is also implemented.

In [14]:
secret2A = random.randbytes(32)
pubkey2A = ecdh.public_key(secret2A)
secret2B = random.randbytes(32)
pubkey2B = ecdh.public_key(secret2B)
ecdh.ecmqv(secretA, secret2A, pubkeyB, pubkey2B).hex()

'ffc54247e2211387acac4e7e32602bbc61127e40e10c497dd90ac2fff7b6e357'

<a name="custom_curves"></a>
## Custom elliptic curves

In addition to the curves available with `ecc.elliptic_curves.get_curve`, it is
possible to build custom curves. These curves are defined over a finite field $F_p$ for a prime $p \neq 2,\ 3$. Three classes are available:
* `Weierstrass_Curve(p, a, b)`, represent elliptic curves in canonical form $y^2 = x^3 + ax + b\ (\mathrm{mod}\ p)$. This is the most general form, associated to ECDSA.
* `Montgomery_Curve(p, A, B)`, elliptic curves in Montgomery form $By^2 = x^3 + Ax^2 + x\ (\mathrm{mod}\ p)$. This is a subset of Weierstrass curves, for which an efficient algorithm exists for scalar multiplication. These curves are associated to ECDH.
* `Edwards_Curve(p, a, d)`, elliptic curves in Edwards form $ax^2 + y^2 = 1 + dx^2y^2\ (\mathrm{mod}\ p)$. These curves are associated with EdDSA.

These classes implement point addition, scalar multiplication, point encoding, and conversion between different curve forms.

However, algorithms to compute the curve cardinal and find a generator are not implemented. Thus, when using a custom elliptic curve to implement cryptographic protocols, one must provide the base point and group order manually. The cardinal of a curve and its generator can be determined by the following PARI/GP code:
```
gp> E = ellinit([a, b]*Mod(1, p)); \\ Weierstrass form
gp> ellcard(E)
gp> ellgenerators(E)
```

In [15]:
from ecc.elliptic_curves import Weierstrass_Curve, Montgomery_Curve

In [16]:
# Custom curve parameters: brainpoolP256r1 from RFC 5639
p = 0xA9FB57DBA1EEA9BC3E660A909D838D726E3BF623D52620282013481D1F6E5377 # base field prime
a = 0x7D5A0975FC2C3057EEF67530417AFFE7FB8055C126DC5C6CE94A4B44F330B5D9 # y^2 = x^3 + ax + b
b = 0x26DC5C6CE94A4B44F330B5D9BBD77CBF958416295CF7E1CE6BCCDC18FF8C07B6
brainpool_G = (0x8BD2AEB9CB7E57CB2C4B482FFC81B7AFB9DE27E1E3BD23C23A4453BD9ACE3262, # base point
               0x547EF835C3DAC4FD97F8461A14611DC9C27745132DED8E545C1D54C72F046997)
brainpool_n = 0xA9FB57DBA1EEA9BC3E660A909D838D718C397AA3B561A6F7901E0E82974856A7 # order of <G>
brainpool_h = 1 # cofactor, h = Card(EC) / n

brainpool_ec = Weierstrass_Curve(p, a, b) # curve construction

In [17]:
# scalar multiplication: test vector from RFC 7027
d = 0x81DB1EE100150FF2EA338D708271BE38300CB54241D79950F77B063039804F1D
dG = (0x44106E913F92BC02A1705D9953A8414DB95E1AAA49E81D9E85F929A8E3100BE5,
      0x8AB4846F11CACCB73CE49CBDD120F5A900A69FD32C272223F789EF10EB089BDC)

In [18]:
brainpool_ec.mult(brainpool_G, d)

(30786306364684019669845085647834227301026705121148702657850323422577469426661,
 62738119601096463087058618165599972860801258532835385944058084661017583328220)

In [19]:
brainpool_ec.mult(brainpool_G, d) == dG

True

In [20]:
brainpool_ec.add(brainpool_ec.mult(brainpool_G, d-1), brainpool_G)

(30786306364684019669845085647834227301026705121148702657850323422577469426661,
 62738119601096463087058618165599972860801258532835385944058084661017583328220)

In [21]:
brainpool_ec.mult(brainpool_G, brainpool_n), brainpool_ec.identity # n<G> == (inf, inf), the identity

((inf, inf), (inf, inf))

Conversion between curve point and byte strings is implemented according to the corresponding RFC for the three `Elliptic_Curve` subclasses:
* `Weierstrass_Curve`: [SEC 1: Elliptic Curve Cryptography](https://www.secg.org/sec1-v2.pdf) sections 2.3.3 and 2.3.4.
* `Montgomery_Curve`: [RFC 7748](https://www.rfc-editor.org/rfc/rfc7748#section-5) section 5.
* `Edwards_Curve`: [RFC 8032](https://www.rfc-editor.org/rfc/rfc8032#section-5.1) sections 5.1.2 and 5.1.3.

In [22]:
dG_enc = brainpool_ec.encode_point(dG, compress=False)
dG_enc.hex()

'0444106e913f92bc02a1705d9953a8414db95e1aaa49e81d9e85f929a8e3100be58ab4846f11caccb73ce49cbdd120f5a900a69fd32c272223f789ef10eb089bdc'

In [23]:
dG_dec = brainpool_ec.decode_point(dG_enc)
dG_dec

(30786306364684019669845085647834227301026705121148702657850323422577469426661,
 62738119601096463087058618165599972860801258532835385944058084661017583328220)

In [24]:
dG == dG_dec

True

The various representations of elliptic curves are somewhat equivalent. More precisely, there exist [birational equivalences](https://en.wikipedia.org/wiki/Montgomery_curve#Equivalence_with_twisted_Edwards_curves) between the different curve forms:
* Edwards <-> Montgomery
* Montgomery -> Weierstrass
* (in some cases) Weierstrass -> Montgomery

In [25]:
p = 2**255 - 19
ec, G, n, _ = ecc.get_curve('edwards25519')

In [26]:
# Convert edwards25519 to curve25519
ec_ = Montgomery_Curve(*ec.montgomery_form()) # curve25519
G_ = ec.montgomery_point(G)

In [27]:
ec_

Montgomery_Curve(
    p=0x7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffed,
    A=0x76d06,
    B=0x7ffffffffffffffffffffffffffffffffffffffffffffffffffffffffff892e5)

In [28]:
G_

(9,
 46155036877857898950720737868668298259344786430663990124372813544693780678454)

In [29]:
# it does not quite correspond to curve25519
curve25519, G_25519, *_ = ecc.get_curve('curve25519')

In [30]:
curve25519

Montgomery_Curve(
    p=0x7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffed,
    A=0x76d06,
    B=0x1)

In [31]:
G_25519

(9,
 43114425171068552920764898935933967039370386198203806730763910166200978582548)

In [32]:
# but the two can be mapped to each other (see RFC 7748 section 4.1)
sqrt = 0xf26edf460a006bbd27b08dc03fc4f7ec5a1d3d14b7d1a82cc6e04aaff457e06 # sqrt(-486664) (mod p)

In [33]:
G_[1]*sqrt % p == G_25519[1]

True

In [34]:
ec_.B * pow(sqrt, -2, p) % p == curve25519.B

True

In [35]:
# The group structure is nevertheless preserved
k = random.randint(1, n-1)
kG = ec.mult(G, k)

In [36]:
# both in Montgomery form
kG_ = ec.montgomery_point(kG)

ec_.mult(G_, k) == kG_

True

In [37]:
# and in Weierstrass form
ec_ = Weierstrass_Curve(*ec.weierstrass_form())
G_ = ec.weierstrass_point(G)
kG_ = ec.weierstrass_point(kG)

ec_.mult(G_, k) == kG_

True

## <a name="custom_algos"></a>Cryptographic algorithms with custom curves

It is also possible to create ECDSA/EdDSA/ECDH instances with custom curves. For this, the corresponding objects must be instantiated from the classes `ECDSA_Base`, `ECDH`, `EdDSA`. Along with the elliptic curve, these require other parameters such as a subgroup generator, the subgroup order, hash functions (for signatures schemes), etc.

In [38]:
from ecc.cryptography import ECDSA_Base, ECDH

Here, we use the curve brainpoolP256r1 to build a custom ECDH instance.

In [39]:
custom_ecdh = ECDH(brainpool_ec, brainpool_G, brainpool_n, brainpool_h)
# test vector from RFC 7027
secretA = 0x81DB1EE100150FF2EA338D708271BE38300CB54241D79950F77B063039804F1D
secretB = 0x55E40BC41E37E3E2AD25C3C6654511FFA8474A91A0032087593852D3E7D76BD3
pubkeyB = custom_ecdh.public_key(secretB)
test = (0x89AFC39D41D3B327814B80940B042590F96556EC91E6AE7939BCE31F3A18BF2B,
        0x49C27868F4ECA2179BFD7D59B1E3BF34C1DBDE61AE12931648F43E59632504DE)
shared_secret = custom_ecdh.ecdh(secretA, pubkeyB, use_cofactor=False)

In [40]:
shared_secret == custom_ecdh.ec.encode_point(test)

True

...and a custom ECDSA instance. Here, due to the technical aspects of nonce generation, a fixed selection of hash functions is available.

In [41]:
custom_ecdsa = ECDSA_Base(brainpool_ec, brainpool_G, brainpool_n, hash_name="BLAKE-256")

ValueError: incorrect hash name BLAKE-256, available hashes are {'sha1', 'sha3-384', 'md5', 'sha3-512', 'sha256', 'sha384', 'sha3-256', 'sha3-224', 'sha512', 'sha224'}

In [42]:
custom_ecdsa = ECDSA_Base(brainpool_ec, brainpool_G, brainpool_n, hash_name="SHA3-384") # the hash name is casefolded
message = b"Lorem ipsum dolor sit amet"
secret = random.randint(2, ecdsa.n-1)
pubkey = custom_ecdsa.public_key(secret)
signature = custom_ecdsa.sign(message, secret)

In [43]:
signature

(2589051408365795916842484484958834543526871002871676601028520264343784797039,
 36366557545319682578001883516756293663306339157145900635648579834367156324483)

In [44]:
custom_ecdsa.verify(message, signature, pubkey)

True

## <a name="cryptanalysis"></a>Cryptanalysis

Various attacks on elliptic curve cryptography protocols have been described. One of them, nonce reuse on ECDSA, has been illustrated above. Two additional attacks are implemented:
* Pohlig-Hellman algorithm, which solves the discrete logarithm problem;
* Partially known nonces attack, which recovers the secret using information leakage in digital signatures.

See [https://safecurves.cr.yp.to](https://safecurves.cr.yp.to) for detailed info about the intrinsic security of ECC.

### <a name="pohlig_hellman"></a>Pohlig-Hellman algorithm

The security of ECC is based on the hardness of the discrete logarithm problem (DLP): given two curve points $G$ and $xG$, recover the secret $x$.
[Pohlig-Hellman algorithm](https://en.wikipedia.org/wiki/Pohlig%E2%80%93Hellman_algorithm) can be used when the order $n$ of the group $\langle G\rangle$ can be decomposed in small coprime factors: $n = \prod_i q_i$, with $\mathrm{gcd}(q_i, q_j) = 1$ for $i\neq j$. The idea is to solve the DLP on each factor $q_i$ (for instance with Pollard's rho method) and reconstruct the value using chinese remainder theorem.

This attack works only if the factors $q_i$ are small, which is obviously not the case for standard curves.

In [45]:
from ecc.cryptanalysis import pohlig_hellman, point_order

In [46]:
# Flawed curve parameters found using PARI/GP
p = 0x9afeae1e4e03e80d0ef2cfe58bb4bad1c88d02b2f527e8f1
a, b = 19, 22
card = 3800462524824166917141813802514024826521312670317078329372
G = (427309928217811500732851283822615328666369400468094808752,
      3071918774523334425905798193770162633643081517753727914870)

ec = Weierstrass_Curve(p, a, b)
n, n_factors = point_order(G, ec, card)

In [47]:
n, n_factors # multiple `small` factors

(3800462524824166917141813802514024826521312670317078329372,
 {2: 2,
  29: 2,
  6529: 1,
  73637: 1,
  1114273: 1,
  6335533: 1,
  15559: 1,
  3722009: 1,
  765590759: 1,
  7507704399191: 1})

In [52]:
# Pohlig-Hellman attack, takes a bit of time
sk = random.randint(2, n-1)
pk = ec.mult(G, sk)
s = pohlig_hellman(G, pk, ec, n, n_factors) # solving discrete logarithm with Pollard's rho on each coprime factor

s == sk

True

### <a name="hnp"></a>Partially known nonces attack

This attack allows to recover an ECDSA/EdDSA secret from the knowledge of some LSBs of the nonces used in ECDSA signatures.

An ECDSA signature is a couple $(r, s)$ such that $xr = sk - h\ (\mathrm{mod}\ q)$, with
* $h$ the hash of the signed message (known);
* $k$ the nonce, of which we know LSBs: $k =  = b*2^l + a$ with $a$ known;
* $x$ the secret to determine.

The problem of recovering $x$ above can be formulated as a hidden number problem $|(tx - u)\ (\mathrm{mod}\ q)| < 2^{l+1}$, with
* $t = 2^{-l}rs^{-1}\ (\mathrm{mod}\ q)$;
* $u = 2^{-l}(a - hs^{-1})\ (\mathrm{mod}\ q) + q/2^{l+1}$.
This problem can in turn be solved using LLL reduction and Babai's neareest plane alogorithm.

For more details, see
* [The Insecurity of the Elliptic Curve Digital Signature Algorithm with Partially Known Nonces](https://www.researchgate.net/publication/2406712_The_Insecurity_of_the_Elliptic_Curve_Digital_Signature_Algorithm_with_Partially_Known_Nonces) - Nguyen and Shparlinski
* [Mathematics of Public Key Cryptography](https://www.math.auckland.ac.nz/~sgal018/crypto-book/crypto-book.html) - Steven Galbraith, chapters 18 and 21.7
      
Although EdDSA is also subject to the attack, the implementation provided here is specific to ECDSA. The EdDSA protocol differs slightly from ECDSA, causing some differences in the attack, more specifically for the translation to the hidden number problem. Moreover, with EdDSA, it is not possible to recover the true secret, but only the derived secret (which is nevertheless enough to forge signatures).

In [48]:
from ecc.cryptanalysis import signatures_to_hnp, partially_known_nonces_attack

In [49]:
ecdsa = ecc.ECDSA(curve_name="Secp256k1", hash_name="sha256")
secret = random.randint(2, ecdsa.n-1)

# Attack setup
nb_lsb_known = 9 # number of known LSBs
nb_signatures = (3*ecdsa.n.bit_length() + nb_lsb_known-1) // nb_lsb_known

In [50]:
# Build a random instance for partially known nonces attack
messages = [random.randbytes(random.randint(0, 80)) for _ in range(nb_signatures)] # random messages
hashes = [ecdsa.msg2int(m) for m in messages] # corresponding ECDSA hashes `h`
nonces = [random.randint(2, ecdsa.n-1) for _ in range(nb_signatures)] # random nonces `k`
known_values = [k % 2**nb_lsb_known for k in nonces] # extract LSBs from nonces, the `a`s
signatures = [ecdsa.sign(msg, secret, k) for msg, k in zip(messages, nonces)] # corresponding signatures (r, s)

In [51]:
# Partially known nonces attack
hnp_pairs = signatures_to_hnp(signatures, ecdsa.n, nb_lsb_known, known_values, hashes)
recovered_secret = partially_known_nonces_attack(hnp_pairs, ecdsa.n, nb_lsb_known)

recovered_secret == secret

True

## <a name="tests"></a>Running tests

Finally, some tests are provided, mainly test vectors from various RFCs and websites.
The output is a bit verbose...

In [ ]:
from ecc.tests import test_ec_algebra, test_ec_cryptography, test_ec_cryptanalysis

test_ec_algebra(curve_operations=True,
                codecs=True,
                curve_equivalences=True,
                birational_equivalences=3)

test_ec_cryptography(ecdsa=True,
                      eddsa=True,
                      ecdh=True)

test_ec_cryptanalysis(pohlig_hellman=True,
                      partially_known_nonces=2)